In [16]:
import sys
import os
sys.path.append(os.path.abspath('../../../../../STUDY/CRAWLING'))
from SeleniumScraper import SeleniumScraper

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from tqdm import tqdm


import pandas as pd
import numpy as np
import datetime
import pickle
import time
import re

In [6]:
def extract_review_details(li_element):
    
    """ Extract Review Deatails """
    
    review_dict = {}
    try:
        ############################# 각 요소 ############################# 
        date = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'timestamp_reviewDate__dsF9n'))).text
        rating = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-rating_ratingLabel__0_Hk9'))).text
        one_line_review = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'heading_Heading__BqX5J.heading_Level3__X81KK'))).text
        job = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-avatar_avatarLabel__P15ey'))).text
        history = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'text-with-icon_TextWithIcon__5ZZqT'))).text
        
        try:
            region = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-avatar_tagsContainer__9NCNs'))).text.split('\n')[1]
        except:
            region=None
        
        pros = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test="review-text-PROS"]'))).text
        cons = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test="review-text-CONS"]'))).text
    
        try:
            show_more_button_class_nm = 'expand-button_ExpandButton__Wevvg'
            show_more_button = WebDriverWait(li_element, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, show_more_button_class_nm))
            )
            show_more_button.click()
    
            feedback = WebDriverWait(li_element, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test="review-text-FEEDBACK"]'))
            ).text
        except:
            feedback = None
        
        review_dict.update({
            'Date': date,
            'Job': job,
            'History': history,
            'Region': region if region else None,
            'One_line_review': one_line_review,
            'Rating': rating,
            'Pros': pros,
            'Cons': cons,
            'Feedback': feedback# if feedback else None,
        })

        ############################# Sub Rating ############################# 
        try:
            rating_element = li_element.find_element(By.CLASS_NAME, 'review-details-bar_reviewRatingAndFeaturedContainer__J8iG9')
            time.sleep(2)
            
            # ActionChains로 마우스를 이동시켜서 숨겨진 요소 표시
            ActionChains(driver).move_to_element(rating_element).perform()
            tooltip = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-test="review-subratings-tooltip"]'))
            )
            # 숨겨진 요소의 요소 전부 값 가져오기
            tooltip_html = tooltip.get_attribute("innerHTML")
            soup = BeautifulSoup(tooltip_html, 'html.parser')
            
            # 각 항목을 포함하는 div를 찾음
            sub_ratings = soup.find_all('div', class_='review-rating_subRating__0Q_Z0')
            
            # 각 항목별로 점수 계산
            for rating in sub_ratings:
                # 항목 이름 가져오기
                category = rating.find('span', class_='review-rating_subRatingText__Wn3AL').text.strip()
                # --outline-percentage가 0%인 별만 카운트
                stars = len(rating.find_all('div', style="--outline-percentage: 0%;"))# 별 개수를 점수로 계산
                review_dict[category] = stars
                
        except Exception:
            pass;

        ############################# Check List #############################
        try:
            check_list = li_element.find_elements(By.CLASS_NAME, 'review-details_experienceContainer__2W06X')

            # 각 요소의 전체 HTML 출력
            for element in check_list:
                html_content = element.get_attribute("outerHTML")
                soup = BeautifulSoup(html_content, 'lxml')

                # 클래스 마지막 부분에 따른 값 매핑 딕셔너리
                class_value_map = {
                    'LGHYG': 'V',
                    'BCbQd': 'empty',
                    'jmOo8': '-',
                    '0GqA8': 'X'
                }

                # 각 span 요소의 텍스트와 해당 클래스명 끝자리에 따른 값 저장
                for span in soup.find_all('span'):
                    # span 텍스트 가져오기
                    span_text = span.text.strip()
                    # 부모 div 요소의 마지막 클래스명 부분 가져오기
                    parent_div = span.find_parent('div')
                    last_class_part = parent_div['class'][-1].split('_')[-1]
                    # 매핑된 값을 result 딕셔너리에 추가
                    review_dict[span_text] = class_value_map.get(last_class_part, 'Unknown')
        
        except Exception:
            pass;
    
    except Exception as e:
        print(f"Error extracting review details: {e}")
    
    return review_dict, date

In [7]:
# 전체 회사 목록
with open('all_company_list.pickle', 'rb') as f:
    company_list = pickle.load(f)
# 완료된 회사 목록
with open('complete_company_list.pickle', 'rb') as f:
    complete_company_list = pickle.load(f)

# 남은 회사 목록
extra_companies = [company for company in company_list if company not in complete_company_list]

In [8]:
# 클래스 초기화
scraper = SeleniumScraper(
    search_url="https://www.glassdoor.com/Reviews/index.htm?overall_rating_low=3.5&page=1&locId=1147401&locType=C&occ=Data%20Scientist"
)

review_df = pd.DataFrame()
result_df = pd.DataFrame()

for company in extra_companies:
    
    print(f"Processing company: {company}")
    
    file_name = f"reviews_{company}.csv"
    # 검색
    search_object = '//*[@id="companyAutocomplete-companyDiscover-employerSearch"]'  # 검색 입력창 XPath
    button_object = '//*[@id="Explore"]/div[2]/div/div/div[2]/button'  # 검색 버튼 XPath
    scraper.get_search(company, search_object, button_object)
    
    # 검색 결과에서 첫 번째 링크 클릭
    search_list_xpath = '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]'
    driver = scraper.get_driver() # driver 객체 가져오기
    first_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, search_list_xpath))
    )
    first_result_link = first_result.find_elements(By.CSS_SELECTOR, 'a')[0].get_attribute('href')
    driver.get(first_result_link)

    # 리뷰 페이지로 이동
    a_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#reviews > a')))
    review_href = a_element.get_attribute('href')
    driver.get(review_href)

    # 날짜 기준 정렬
    sort_date_parent_class_nm = 'sort_SortButton__hzneq'
    sort_date_child_datatest = '[data-test="select-dropdown-value-DATE"]'
    scraper.filterlng(sort_date_parent_class_nm, sort_date_child_datatest)

    try:
        # 기존 filterling clear
        clear_parent_class_nm = 'filter-menu_FilterMenuButton__NOhT8'
        clear_child_datatest = '[data-test="filter-menu-clear-button"]'
        scraper.filterlng(clear_parent_class_nm, clear_child_datatest)
    except:
        with open('filtering_fail_list.pickle', 'rb') as f: # Read
            filtering_fail_list = pickle.load(f)
            filtering_fail_list.append(company) # Append
            
        with open("filtering_fail_list.pickle","wb") as f: # Write
            pickle.dump(filtering_fail_list, f)
        print('filterling button not found')
    
    # last page search
    last_p_elements = driver.find_elements(By.CSS_SELECTOR, "p.pagination_PageNumberText__zy_hr")
    last_page = int([p.text for p in last_p_elements][-1])
    
    # pagination
    iteration_count = 0
    for page in tqdm(range(1, last_page + 1)):
        iteration_count+=1
        if page > 1:
            review_url = review_href.replace('.htm', f'_P{page}.htm')
            driver.get(review_url)
        else:
            pass;

        try:
            li_elements = driver.find_elements(By.XPATH, '//*[@id="ReviewsFeed"]/ol/li') #?
            for li_element in li_elements:
                review_dict, date = extract_review_details(li_element)
                
                # 2021.01.01 이전이면 종료
                # current_date = datetime.datetime.strptime(current_date_str, "%b %d, %Y")
                # target_date = datetime.datetime(2021, 1, 1)
                # if current_date < target_date:
                #     break;
                    
                review_dict['Company'] = company
                review_df = pd.concat([review_df, pd.DataFrame([review_dict])], ignore_index=True)

            if iteration_count%100:
                review_df.to_csv(f'tmp_company_review/{file_name}', index=False)
                
        except Exception as e:
            print(f"Error processing reviews on page {page} for {company}: {e}")
    
    result_df = pd.concat([result_df, review_df], ignore_index=True)
    result_df.to_csv('company_review_data/complete_'+file_name, index=False)
    
    with open('complete_company_list.pickle', 'rb') as f:
        complete_company_list = pickle.load(f)
    complete_company_list.append(company)
    
    with open("complete_company_list.pickle","wb") as f:
        pickle.dump(complete_company_list, f)
    
    print(f"{company} complete !")

print("Scraping complete")

The chromedriver version (130.0.6723.69) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.140); currently, chromedriver 131.0.6778.108 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


Processing company: Lithia Motors, Inc.
filterling button not found


  1%|▌                                           | 1/86 [00:23<33:55, 23.95s/it]

Error processing reviews on page 1 for Lithia Motors, Inc.: name 'current_date_str' is not defined


  2%|█                                           | 2/86 [00:59<43:29, 31.07s/it]

Error processing reviews on page 2 for Lithia Motors, Inc.: name 'current_date_str' is not defined


  3%|█▌                                          | 3/86 [01:20<37:06, 26.82s/it]

Error processing reviews on page 3 for Lithia Motors, Inc.: name 'current_date_str' is not defined


MaxRetryError: HTTPConnectionPool(host='localhost', port=57382): Max retries exceeded with url: /session/098fb257ad8cb930fccf75eec7022332/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11ac1ffa0>: Failed to establish a new connection: [Errno 61] Connection refused'))